# Weighted constraints
## Exploring the exact conditions for optimal feedforward

This notebook shows an (artificial) case where the feedforward is not properly generated: $\frac{\partial^2 e}{\partial q_i \partial t} \neq 0$, and for some i, j : $  ~ J_i \neq J_j $.

The case is however to complicated to give an explicit expression for the resulting robot trajectory.  Ttherefore we compare the case with low gain and feedforward to the case with high gain and no feedforward (due to the changing weights).


In [ ]:
%matplotlib inline
import numpy as np
from etasl_py.etasl import etasl_simulator,array_to_dict,dict_to_array,to_deg, to_rad
from bokeh.plotting import figure, show, output_notebook
from bokeh.layouts import gridplot
from etasl_py.bokehplots import plotv, append_units

output_notebook()

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; } </style>"))

## Constraints that cause the non-optimal feed-forward

The task definition is split up in three parts.  The first part contains the robot defintion:

In [ ]:
etasl_robot="""
    require("context")
    require("geometric")
    -- Robot:
    local u=UrdfExpr();
    local fn = rospack_find("etasl_py_examples").."/robots/ur10_robot.urdf"
    u:readFromFile(fn)
    u:addTransform("ee","tool0","base_link")
    local r = u:getExpressions(ctx)
    ee = r.ee
    robot_jname={"shoulder_pan_joint","shoulder_lift_joint","elbow_joint","wrist_1_joint","wrist_2_joint","wrist_3_joint"}
    robot_jval = {}
    for i=1,#robot_jname do
       robot_jval[i]   = ctx:getScalarExpr(robot_jname[i])
    end
"""

The next part contains the trajectories (tgt_x,tgt1_y,tgt_z) and (tgt_x,tgt2_y,tgt_z).  It  is relatively low at $2 s^{-1}$.

In [ ]:
etasl_traj1 = """
    -- Task:
    control_gain = 2
    tgt_x  = sin(time)*0.15 + 0.7
    tgt_z  = 0
    tgt1_y = sin(2.5*time)*0.1 +0.4
    tgt2_y = sin(2*time)*0.1 + 0.1
    w1 = 1-sin(0.5*time)*0.8
    w2 = 1+cos(0.5*time)*0.9
    
"""

The last part contains the task definition (the similar as the previous definitions a laserspot following a trajectory, but with some
time dependent modification to the constraint using 'tgt2_y').

In [ ]:
etasl_cnstr = """
    d = Variable{context=ctx, name="d", vartype="feature"}

    Constraint{
        context=ctx,
        name="laserdistance",
        expr=d,
        target_lower = 0.3,
        target_upper = 0.45,
        K = control_gain
    }

    laserspot = ee*vector(0,0,d)

    Constraint{
        context=ctx,
        name="x",
        expr = tgt_x - coord_x(laserspot),
        priority = 2,
        K        = control_gain
    }
    Constraint{
        context=ctx,
        name="z",
        expr = tgt_z - coord_z(laserspot),
        priority = 2,
        K        = control_gain
    }
    
    -- conflicting constraints : 
    Constraint{
        context=ctx,
        name="y",
        expr = (tgt1_y - coord_y(laserspot)),
        priority = 2,
        K        = control_gain,
        weight   = w1
    }
    Constraint{
        context=ctx,
        name="y",
        expr = (tgt2_y - (1+0.5*sin(2*time))*coord_y(laserspot)),
        priority = 2,
        K        = control_gain,
        weight   = w2
    }    
    -- Output:
    ctx:setOutputExpression("error_x",coord_x(laserspot)-tgt_x)
    ctx:setOutputExpression("tgt1_y",tgt1_y)
    ctx:setOutputExpression("tgt2_y",tgt2_y)
    ctx:setOutputExpression("w1",w1)
    ctx:setOutputExpression("w2",w2)
    ctx:setOutputExpression("y",coord_y(laserspot))    
"""


We run the simulation with a high frequency ($1000 Hz$ ) because later on, we want to run the simulation with a very high control gain.

In [ ]:
e1 = etasl_simulator(regularization_factor= 1E-9)
e1.readTaskSpecificationString(etasl_robot + etasl_traj1 + etasl_cnstr)

pos_lbl = ['shoulder_pan_joint','shoulder_lift_joint', 'elbow_joint', 
           'wrist_1_joint', 'wrist_2_joint',  'wrist_3_joint','d']
initial_jpos = np.array([0, -np.pi*0.6 , np.pi*0.6,-2.0,-np.pi*0.5,0.1,0.1])
N=20000
dt=0.001

e1.initialize(initial_jpos, pos_lbl)
e1.simulate(N=N,dt=dt)

After the simulation, we present the results using Bokeh.  This task specification results in fast, but not yet unrealisticly fast, motions.

In [ ]:
plotv(e1.TIME, e1.OUTP, append_units(e1.OUTP_LBL[0:3]," [m]")+["w1","w2","y [m]"], ncols=3,plotw=400,ploth=250)

To check whether there is a tracking error, we compare to the same specification, but with a extremely high control gain (only possible in simulation), and with the feedforward mechanism for changing weights disabled (using `make_constant`)

In [ ]:
etasl_traj2="""
    -- Task:
    control_gain = 400
    tgt_x  = sin(time)*0.15 + 0.7
    tgt_z  = 0
    tgt1_y = sin(2.5*time)*0.1 +0.4
    tgt2_y = sin(2*time)*0.1 + 0.1
    w1 = make_constant(1-sin(0.5*time)*0.8)
    w2 = make_constant(1+cos(0.5*time)*0.9)
"""

We run a simulation with the same part for the robot `etasl_robot` and the constraints `etasl_cnstr` as before:

In [ ]:
e2 = etasl_simulator(regularization_factor= 1e-9)
e2.readTaskSpecificationString(etasl_robot + etasl_traj2 + etasl_cnstr)
e2.initialize(initial_jpos, pos_lbl)
e2.simulate(N=N,dt=dt)

In [ ]:
print("control errors : ")
plotv(e2.TIME, e2.OUTP, append_units(e2.OUTP_LBL[0:3]," [m]")+["w1","w2","y [m]"], ncols=3,plotw=400,ploth=250)

We can now compare the results of both simulations and see whether this results in a tracking error:

In [ ]:
plotv(e2.TIME,e2.OUTP[:,5:6]-e1.OUTP[:,5:6],["difference [m]"],ncols=2,plotw=800,ploth=350)

And indeed this leeds to a (still relatively small) **tracking error**.